In [1]:
import pandas as pd
import numpy as np
from astropy.io import fits
import os
from tqdm import tqdm

# Load the CSV file with redshift values
data = pd.read_csv('filtered_non_spectral_data.csv')

# Directory containing the spectra files and where to save the corrected spectra
spectra_dir = 'spectra1'
corrected_spectra_dir = 'redshift_corrected_spectra'
os.makedirs(corrected_spectra_dir, exist_ok=True)

# Function to apply redshift correction
def correct_redshift(wavelength, z):
    return wavelength / (1 + z)

# Iterate through each entry in the CSV and process the corresponding FITS file
for _, row in tqdm(data.iterrows(), total=data.shape[0]):
    plate = int(row['Plate'])
    mjd = int(row['MJD'])
    fiber = int(row['Fiber'])
    z = row['z']

    # Construct the file name
    filename = f'spec-{plate:04d}-{mjd:05d}-{fiber:04d}.fits'
    file_path = os.path.join(spectra_dir, filename)
    corrected_file_path = os.path.join(corrected_spectra_dir, filename)

    # Check if the spectrum file exists
    if os.path.exists(file_path):
        with fits.open(file_path) as hdul:
            # Assuming the wavelength is in logarithmic scale and needs to be corrected
            loglam = hdul[1].data['loglam']
            flux = hdul[1].data['flux']
            
            # Correct the wavelength for redshift
            wavelength = 10**loglam
            corrected_wavelength = correct_redshift(wavelength, z)

            # Save the corrected data; here we just update the flux for demonstration
            # In practice, you might need to update the structure and save as per your requirement
            hdul[1].data['loglam'] = np.log10(corrected_wavelength)
            hdul.writeto(corrected_file_path, overwrite=True)
    else:
        print(f'File {filename} not found.')


100%|█████████████████████████████████████| 28837/28837 [33:27<00:00, 14.37it/s]
